<a href="https://colab.research.google.com/github/JordanDCunha/On-Complexity/blob/main/Chapter11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 11.1 Introduction — Agent-Based Models

## What Are Agent-Based Models (ABMs)?
- Models composed of **agents** that:
  - Gather information
  - Make decisions
  - Take actions
- Agents represent people or other decision-making entities

## Key Characteristics of Agents
- Situated in **space or networks**
- Interact **locally** with nearby agents
- Have **imperfect or incomplete information**
- Often **heterogeneous** (agents differ from one another)
- Frequently include **randomness** in agents or environment

## How ABMs Differ from Rule-Based Models
- Earlier models: identical components + fixed rules
- ABMs: diverse agents + adaptive, decision-driven behavior

## Why Agent-Based Models Matter
- Useful for studying **non-equilibrium dynamics**
- Can also analyze **equilibrium behavior**
- Help link:
  - **Individual decisions**
  - **Emergent system-level outcomes**

## Fields of Application
- Economics
- Social sciences
- Natural sciences
- Widely used since the **1970s**

## Code Reference
- Chapter code located in `chap11.ipynb`
- See Section 1.4 for instructions on using the code


# 11.2 Schelling’s Model of Segregation

## Background
- Proposed by **Thomas Schelling (1969)** in *“Models of Segregation”*
- Designed to study how **segregation can emerge** from simple individual preferences

## Model Setup
- World is a **grid**
  - Each cell = a **house**
- Houses contain:
  - **Red agents**
  - **Blue agents**
  - ~**10% empty houses**
- Roughly equal numbers of red and blue agents

## Neighborhood Definition
- Each agent considers the **8 surrounding cells** (Moore neighborhood)

## Agent Happiness Rule
- Agent is **happy** if:
  - At least **2 neighbors are like itself**
- Agent is **unhappy** if:
  - It has **0 or 1 similar neighbors**

## Movement Rule
- At each step:
  - Pick a **random agent**
  - If happy → **do nothing**
  - If unhappy → **move to a random empty cell**

## Emergent Behavior
- Starting from a **random configuration**:
  - Small clusters of similar agents form quickly
  - Clusters **grow and merge**
  - Final state has:
    - Few large clusters
    - Mostly **homogeneous neighborhoods**

## Key Insight
- Strong segregation emerges even though:
  - Agents are **not strongly biased**
  - Agents are comfortable with **mixed neighborhoods**
  - They only avoid being **heavily outnumbered**

## Interpretation
- Observed segregation **does not imply individual racism**
- System-level patterns can arise from:
  - Mild individual preferences
  - Local interactions

## Important Limitation
- The model shows a **possible cause** of segregation
- It does **not identify actual causes** in real societies
- Real-world racism and segregation are **far more complex**


# 11.3 Implementation of Schelling’s Model


In [ ]:
class Schelling(Cell2D):

    def __init__(self, n, p):
        self.p = p
        choices = [0, 1, 2]
        probs = [0.1, 0.45, 0.45]
        self.array = np.random.choice(choices, (n, n), p=probs)


## Cell States
- `0` → empty
- `1` → red agent
- `2` → blue agent


In [ ]:
a = self.array
red = a == 1
blue = a == 2
empty = a == 0


## Neighbor Counting (8-cell neighborhood)
- Uses convolution to count nearby agents
- Wrap-around boundaries


In [ ]:
options = dict(mode='same', boundary='wrap')

kernel = np.array([[1, 1, 1],
                   [1, 0, 1],
                   [1, 1, 1]], dtype=np.int8)

num_red = correlate2d(red, kernel, **options)
num_blue = correlate2d(blue, kernel, **options)
num_neighbors = num_red + num_blue


## Fractions of Neighbor Types


In [ ]:
frac_red = num_red / num_neighbors
frac_blue = num_blue / num_neighbors


## Fraction of Similar Neighbors
- Red agents compare against red neighbors
- Blue agents compare against blue neighbors
- Empty cells ignored


In [ ]:
frac_same = np.where(red, frac_red, frac_blue)
frac_same[empty] = np.nan


## Identifying Unhappy Agents
- Agent is unhappy if fraction of similar neighbors is **less than** `p`


In [ ]:
unhappy = frac_same < self.p


## Utility Function for Locations


In [ ]:
def locs_where(condition):
    return list(zip(*np.nonzero(condition)))


## Locations of Agents and Empty Cells


In [ ]:
unhappy_locs = locs_where(unhappy)
empty_locs = locs_where(empty)
num_empty = np.sum(empty)


## Agent Movement Rule
- Unhappy agents move to a random empty cell
- Source becomes empty


In [ ]:
for source in unhappy_locs:
    i = np.random.randint(num_empty)
    dest = empty_locs[i]

    a[dest] = a[source]
    a[source] = 0
    empty_locs[i] = source


## Key Relationship (Fill in the Blank)
- Happiness is determined by whether `self.p` is **greater** than `frac_same`


# 11.4 Segregation


In [ ]:
grid = Schelling(n=100, p=0.3)
for i in range(10):
    grid.step()


## Simulation Setup
- Grid size: `n = 100`
- Happiness threshold: `p = 0.3`
- Simulation runs for 10 steps


In [ ]:
# Example metric: average fraction of similar neighbors
np.nanmean(frac_same)


## Observed Behavior
- Clusters of similar agents form very quickly
- Small clusters merge into larger homogeneous regions
- Most agents end up in highly segregated neighborhoods


## Degree of Segregation (Figure 11.1)
- Initial configuration: **50%**
- After 2 steps: **65%**
- After 10 steps: **76%**


## Interpretation
- Agents only require a small fraction of similar neighbors to be happy
- Despite low requirements, segregation increases far beyond the minimum
- Individual preferences lead to unexpected system-level outcomes


## Effect of Varying p (Figure 11.2)
- Higher values of `p` lead to higher final segregation
- Segregation increases over time, then levels off


## Steady-State Result
- When `p = 0.4`, steady-state segregation is approximately **82%**
- Most agents have no neighbors of a different color


## Key Takeaway
- Mild individual preferences can produce extreme segregation
- System behavior is not a simple reflection of individual intent


## 11.5 Sugarscape

- Sugarscape is an **agent-based model** introduced in 1996 by Joshua Epstein and Robert Axtell.
- It was designed to study economics and social science questions using an “artificial society”.
- The model is highly versatile and has been adapted for many research topics.
- In its simplest form, Sugarscape represents a **simple economy**.

### Core Idea
- Agents move on a **2-D grid**.
- They collect and store “sugar”, which represents **economic wealth**.
- Sugar availability varies across the grid.
- Agents differ in their ability to find and consume sugar.
- The model is often used to study **wealth distribution and inequality**.

### Grid Structure
- Each grid cell has a **capacity**, the maximum sugar it can hold.
- The original setup includes:
  - Two high-sugar regions with capacity 4
  - Surrounding concentric regions with capacities 3, 2, and 1

### Agents
- Initially, **400 agents** are placed randomly.
- Each agent has three attributes:
  - **Sugar**: Initial wealth between 5 and 25 units
  - **Metabolism**: Sugar consumed per step (1–4 units)
  - **Vision**: Distance the agent can see in each direction (1–6 cells)

### Agent Behavior (Per Time Step)
- Agents act one at a time in random order.
- Each agent:
  - Scans cells in the four compass directions up to its vision range
  - Chooses the unoccupied cell with the most sugar
  - Breaks ties by choosing the closest cell, then randomly if needed
  - Moves to the chosen cell and harvests its sugar
  - Consumes sugar based on metabolism
  - Dies (starves) if sugar drops below zero

### Environment Dynamics
- After all agents move:
  - Cells regrow sugar, typically 1 unit per step
  - Sugar growth is capped by each cell’s capacity

### Emergent Behavior
- Agents move toward high-sugar regions.
- High-vision agents move faster and survive more easily.
- Low-vision or high-metabolism agents are more likely to starve.
- Initial population drops rapidly, then stabilizes around **250 agents**.
- Surviving agents tend to accumulate large amounts of sugar indefinitely.
- The model demonstrates how **inequality emerges naturally** from simple rules.

### Key Takeaways
- Sugarscape is a model of a simple economy.
- It operates on a 2-D grid.
- It is commonly used to study wealth distribution.
- Agents are heterogeneous in vision, metabolism, and wealth.


## 11.6 Wealth Inequality

- In its basic form, Sugarscape models a **simple ecology**.
- It can be used to study how parameters like sugar growth rate and agent attributes affect:
  - Carrying capacity (steady-state population size)
  - Survival of agents over time
- The model also represents a form of **natural selection**, where agents with higher fitness are more likely to survive.

### Wealth Inequality in Sugarscape
- Agents accumulate sugar at different rates.
- This leads to **wealth inequality**, where some agents become much wealthier than others.
- In the basic model, wealth distribution is **not stationary**:
  - The distribution keeps changing over time
  - No steady-state wealth distribution emerges

### Introducing Finite Lifespans
- To obtain a stationary wealth distribution, agents are given **finite lifespans**.
- Each agent:
  - Has an age that increases every time step
  - Is assigned a random lifespan between 60 and 100 steps
  - Dies when its age exceeds its lifespan
- Agents can also die from starvation.
- When an agent dies:
  - It is replaced by a new agent with random attributes
  - The total population remains constant

### Stationary Wealth Distribution
- Starting with about **250 agents** (near carrying capacity), the model is run for 500 steps.
- The **cumulative distribution function (CDF)** of wealth is measured every 100 steps.
- After about 200 steps:
  - The wealth distribution stabilizes
  - Further changes are minimal

### Observed Wealth Patterns
- The distribution of wealth is **right-skewed**.
- Most agents accumulate relatively little sugar:
  - 25th percentile ≈ 10
  - Median ≈ 20
- A smaller number of agents accumulate much more:
  - 75th percentile ≈ 40
  - Maximum values exceed 150

### Log-Scale Interpretation
- On a log-x scale, the wealth distribution resembles a **Gaussian (normal)** shape.
- This suggests a **lognormal distribution**, which is heavy-tailed.
- Heavy-tailed wealth distributions are common in real-world economies.

### Key Insights
- Sugarscape does not fully explain why wealth distributions are heavy-tailed.
- However, persistent inequality appears across many Sugarscape variants.
- This suggests that **inequality may be a natural outcome** of even very simple economic systems.
- Experiments with taxation and redistribution rules indicate that:
  - Inequality is difficult to eliminate or significantly reduce


# 11.7 Implementing Sugarscape

- Sugarscape is more complex than earlier models.
- The full implementation is not shown in the text.
- Instead, the chapter outlines the **code structure**.
- Complete details are available in `chap11.ipynb`.

Core idea:
- Agents interact with an environment (`Sugarscape`)
- Each step includes movement, harvesting, aging, and possible death


In [ ]:
# 11.7 Implementing Sugarscape

- Sugarscape is more complex than earlier models.
- The full implementation is not shown in the text.
- Instead, the chapter outlines the **code structure**.
- Complete details are available in `chap11.ipynb`.

Core idea:
- Agents interact with an environment (`Sugarscape`)
- Each step includes movement, harvesting, aging, and possible death


## Agent Behavior

During each time step, an agent:

- Moves to a new location chosen by the environment
- Harvests sugar at that location
- Consumes sugar based on metabolism
- Ages by one time unit

The agent does **not** decide everything itself:
- It relies on the environment (`env`) for movement and harvesting


## Environment Methods Used by Agents

The `env` object (a Sugarscape instance) provides:

- **look_and_move(loc, vision)**
  - Takes the agent’s location and vision range
  - Returns the visible cell with the most sugar

- **harvest(loc)**
  - Removes and returns the sugar at a given location


In [ ]:
# Sugarscape class (inherits from Cell2D)

class Sugarscape(Cell2D):

    def step(self):

        # loop through agents in random order
        random_order = np.random.permutation(self.agents)

        for agent in random_order:

            # mark current cell as unoccupied
            self.occupied.remove(agent.loc)

            # agent takes one step
            agent.step(self)

            # remove dead agents
            if agent.is_starving():
                self.agents.remove(agent)
            else:
                # mark new cell as occupied
                self.occupied.add(agent.loc)

        # regrow sugar after all agents move
        self.grow()

        return len(self.agents)


## Sugarscape Step Logic

Each simulation step proceeds as follows:

1. Agents are processed in **random order**
2. Each agent:
   - Leaves its current cell
   - Moves, harvests sugar, and ages
3. Starving agents are removed
4. Surviving agents occupy their new cells
5. Sugar regrows in the environment
6. The method returns the number of living agents

Random ordering prevents unfair advantages based on update order


## Important Helper Functions (Not Shown)

The full implementation uses several NumPy-heavy helper functions:

- **make_visible_locs**
  - Builds all cells an agent can see
  - Sorts by distance
  - Randomizes ties
  - Uses `np.random.shuffle` and `np.vstack`

- **make_capacity**
  - Initializes sugar capacity across the grid
  - Uses `indices`, `hypot`, `minimum`, and `digitize`

- **look_and_move**
  - Selects the visible cell with the most sugar
  - Uses `argmax`


## Key Takeaways

- Sugarscape separates **agent logic** from **environment logic**
- Agents are simple; complexity emerges from interaction
- Random ordering and spatial constraints are crucial
- NumPy enables efficient large-scale simulations


# 11.8 Migration and Wave Behavior

## Overview
- Sugarscape is not designed primarily to study movement patterns
- However, agent migration produces **unexpected wave-like behavior**
- These patterns emerge from local rules, not explicit coordination

## Initial Setup
- All agents start clustered in the **lower-left corner**
- High-capacity sugar regions (“peaks”) are located elsewhere on the grid

## Migration Dynamics
- Agents move toward the nearest high-capacity sugar peak
- If the number of agents exceeds what one peak can support:
  - Sugar is quickly depleted
  - Agents are forced into lower-capacity regions

## Role of Vision
- Agents with **longer vision** can:
  - Cross low-sugar valleys between peaks
  - Reach distant high-capacity regions sooner
- These agents initiate movement toward the northeast

## Wave Formation
- Moving agents leave behind **stripes of empty cells**
- Other agents cannot follow immediately because sugar is depleted
- Once sugar regrows, new groups move forward
- This creates **discrete waves of migration**

## Wave Characteristics
- Each wave behaves like a **coherent object**
- Similar to:
  - Spaceships in Rule 110
  - Moving patterns in the Game of Life
- Waves move **diagonally**, even though:
  - Individual agents only move north or east

## Emergent Behavior
- The diagonal wave motion is **not a property of individual agents**
- It emerges from collective interaction and environmental feedback
- This illustrates a key idea in agent-based models:
  - Groups can exhibit behaviors that individuals do not have

## Key Takeaway
- Migration in Sugarscape demonstrates **emergent, wave-like behavior**
- Complex spatial patterns can arise from simple local rules
- Such aggregate behaviors are common in agent-based systems


# 11.9 Emergence

## Core Idea
- **Emergence** is a central concept in complexity science
- An emergent property arises from **interactions among components**, not from the components themselves

## What Emergence Is *Not*
- Properties that directly follow from component properties are **not emergent**
  - Example: a brick wall is hard because bricks are hard
- Structures built from flexible components (e.g., rigid frames) show only **weak emergence**
  - Their behavior follows well-known physical laws

## Strong Emergence in Models
- **Schelling’s segregation model**
  - Segregation emerges even though agents are not racist
  - System-level outcome differs from individual intentions
- **Sugarscape wealth distribution**
  - Possibly emergent, but weaker
  - Can be partly predicted from agent attributes (vision, metabolism, lifespan)
- **Wave behavior in Sugarscape**
  - Stronger example of emergence
  - Waves move diagonally even though agents cannot

## Key Characteristics of Emergent Properties
- They are **surprising**
- Difficult or impossible to predict from rules alone
- Knowing all the rules does **not guarantee predictability**

## Computational Irreducibility
- Many cellular automata are **computationally irreducible**
- There are **no shortcuts** to predicting their behavior
- The only way to know the outcome is to **run the system**

## Limits of Prediction
- Classical science assumes rules allow prediction
- Complex systems often lack analytic (closed-form) solutions
- Simulations require computation time proportional to the time simulated
- Beyond some point, reliable prediction becomes impossible

## Philosophical Debate
- Some argue emergence reflects **temporary ignorance**
- Others argue some emergent properties can **never** be reduced
- Computational equivalence suggests limits to reductionist explanations

## Takeaway
- Emergence challenges the idea that all systems can be fully explained
- In complex systems, some behaviors may be **fundamentally unpredictable**


# 11.11 Glossary

## Agents
- Entities that model people or decision-making units
- Gather information, make decisions, and take actions
- Usually interact locally in space or networks
- Have imperfect or incomplete information
- Can differ from one another
- Often include randomness

## Agent-Based Models
- Computational models that simulate actions and interactions of autonomous agents
- Used to study how individual behaviors affect system-level outcomes

## Computationally Irreducible
- Systems with no predictive shortcuts
- The only way to determine outcomes is to run the simulation

## Emergent Property
- A system-level characteristic
- Arises from interactions among components
- Not explained by properties of individual components alone

## Metabolism
- Amount of sugar an agent must consume each time step
- Typically chosen uniformly between 1 and 4

## Schelling’s Model
- Grid-based model of housing
- Cells represent houses occupied by red agents, blue agents, or left empty
- About 10% of houses are empty
- Agents evaluate happiness based on their 8 neighbors
- Agents are unhappy if they have too few similar neighbors

## Sugar
- Represents wealth in Sugarscape
- Each agent starts with an initial endowment between 5 and 25 units

## Sugarscape
- Agent-based model of an artificial society
- Used to study economics, wealth distribution, and inequality

## Vision
- Determines how far an agent can see in the grid
- Agents move toward cells with the most sugar
- Vision varies among agents
